#### Import Python packages 

In [3]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [4]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [5]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# creating an event data csv file called event_datafile_new csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


#### Creating a Cluster

In [6]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)
from cassandra.cluster import Cluster
cluster = Cluster(['127.0.0.1'])

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [7]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [8]:
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

## Query 1:

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

### session_library
#### This query is insterested in the exact song that was played in a particular session and item in session. Thus naming the table session_library. The data for the session_library table is partitioned by session_id so session_id acts as the partition key. Each partition shall store data for one particualr session. But session_id doesn't make each record in the data unique. Thus we can use item_in_session column as the clustering column. This primary key will make sure that we will only get one record given the session and item in session ids.



In [9]:
query = "CREATE TABLE IF NOT EXISTS session_library "
query = query + "(session_id int, item_in_session int, artist text, song_title text, song_length float, PRIMARY KEY (session_id, item_in_session))"
try:
    session.execute(query)
except Exception as e:
    print(e) 

#### Insert sessionId, itemInSession, artist, song and length information from the csv file into the session_library table.

In [10]:
file = 'event_datafile_new.csv'
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO session_library (session_id, item_in_session, artist, song_title, song_length)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### Since we partitioned by session_id and clustered by item_in_session, the query returns one unique record for the artist, song and song length the user was listening to in the particular session and the particular item in session. 



In [11]:
query = "select artist, song_title, song_length from session_library where session_id = 338 and item_in_session = 4"
try:
    rows = session.execute(query)
except Exception as e:
        print(e)
        
for row in rows:
    print (row.artist, row.song_title, row.song_length)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


## Query 2:

### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

### user_activity
#### This query is insterested in the songs played by a user in a particular session. Thus naming the table user_activity. The data for the user_activity table is partitioned by user_id so user_id acts as the partition key. Each partition shall store data for one particualr user. But user_id doesn't make each record in the data unique. Thus we can use session_id and item_in_session as the clustering column. Since the query requires data sorted by item in session, we order the cluster by item_in_session. This primary key will make sure that we will only get one record given the user, session and item in session ids. But we can search user activity by user id by session.


In [12]:
query = "CREATE TABLE IF NOT EXISTS user_activity "
query = query + "(user_id int, session_id int, item_in_session int, user text, artist text, song_title text, PRIMARY KEY ((user_id, session_id), item_in_session))"
query = query + "WITH CLUSTERING ORDER BY (item_in_session asc)"
try:
    session.execute(query)
except Exception as e:
    print(e)

#### Insert userId, sessionId, itemInSession, user full name, artist, and song information from the csv file into the user_activity table.

In [14]:
file = 'event_datafile_new.csv'
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO user_activity (user_id, session_id, item_in_session, user, artist, song_title)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[1] + ' ' + line[4], line[0], line[9]))

#### Since we partitioned by user_id and clustered by session_id and item_in_session, the query returns all the records for the artist song and the user's full name for the songs the user was listening to in the particular session. 

In [15]:
query = "select artist, song_title, user from user_activity where user_id = 10 and session_id = 182"
try:
    rows = session.execute(query)
except Exception as e:
        print(e)
        
for row in rows:
    print (row.artist, row.song_title, row.user)

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


## Query 3:

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

### song_library
#### This query is insterested in the user group that listens to a particular song. Thus naming the table song_library. The data for the song_library table is partitioned by song_title so song_title acts as the partition key. Each partition shall store data for one particular song. But song_title doesn't make each record in the data unique. Thus we can use user_id as the clustering column. This primary key will make sure that we will only get one record given the song title and user id. But we can search songs by title and get the list of users that have heard the particular song.

In [17]:
query = "CREATE TABLE IF NOT EXISTS song_library "
query = query + "(song_title text, user_id int, user text, PRIMARY KEY (song_title, user_id))"
try:
    session.execute(query)
except Exception as e:
    print(e)                     

#### Insert song, sessionId, itemInSession, user full name information from the csv file into the song_library table.

In [19]:
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_library (song_title, user_id, user)"
        query = query + " VALUES (%s, %s, %s)"
        session.execute(query, (line[9], int(line[10]), line[1] + ' ' + line[4]))

#### Since we partitioned by song title and clustered by session_id and item_in_session, the query returns all the records for the user's full name who have listened to the given song. 

In [20]:
query = "select user from song_library where song_title = 'All Hands Against His Own'"
try:
    rows = session.execute(query)
except Exception as e:
        print(e)
        
for row in rows:
    print (row.user)

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Drop the tables before closing out the sessions

In [21]:
query = "drop table if exists session_library"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
query = "drop table if exists artist_activity"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
query = "drop table if exists song_library"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [22]:
session.shutdown()
cluster.shutdown()